# Russian language dataset

First, install all prerequisites 

In [ ]:
!pip install langdetect tqdm
!pip install "tqdm==4.43.0"
!sudo apt-get install -y xsltproc
from fastai.basics import *
from tqdm import *
from tqdm.contrib.concurrent import process_map, thread_map
from multiprocessing import Pool
import regex as re
import time
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

NEW_LINE = '<|n|>'

librusec = '/home/u/nas/librusec/lib.rus.ec'
tmpzips = './tmp/zip'
tmptxt = './tmp/txt'
tmpfb2clean = './tmp/fb2clean'
tmpfb2unzip = './tmp/fb2unzip'
data = Path('../data/full')

!mkdir ../data
!mkdir ../data/full
!mkdir ../data/classic
!mkdir tmp
!mkdir tmp/fb2unzip
!mkdir tmp/fb2clean
!mkdir tmp/txt
!mkdir tmp/zip

### Unpack ZIPs

Before running this we need to upload our fb2 zip files to /tmp/zip and then run this script

In [23]:
# clean the output directory 
#!rm -rfv {tmpfb2unzip + '/*'}

In [59]:
# zips = get_files(tmpzips, '.zip')
# print(f'{len(zips)} zip file(s) found')

# def unpack(fn):
#     # replace -o with -n to not overwrite existing files
#     # remove the -q flag for more logging
#     # -j  junk paths (do not make directories)  
#     # -qo flag to remove annoying warning https://www.directadmin.com/features.php?id=2213
#     !unzip -qq -joL -qo -O cp396 {fn} -d {tmpfb2unzip} >>/dev/null

# # Unpack zips in parallel
# thread_map(unpack, zips, max_workers=64)
# print(f'Unzipped all - DONE')

# # Sanitize file and folder names - remove spaces
# !find $tmp -depth -name "* *" -execdir rename 's/ /_/g' "{}" \;

1 zip file(s) found



Unzipped all


### Convert fb2 to txt

In [ ]:
# clean the output directory 
#!rm -rfv {tmpfb2clean + '/*'}

In [ ]:
# clean the output directory 
#!rm -rfv {tmptxt + '/*'}

In [ ]:
# Get fb2s
# fbs = get_files(tmpfb2unzip, '.fb2', recurse=True)

# # Sanitize filenames and move to the 'clean' dir
# for fn in fbs:
#     nn = (str(fn.name)
#         .replace(' ','')
#         .replace('_quot;','')
#         .replace('!','')
#         .replace(',','')
#         .replace('(','')
#         .replace(')','')
#         .replace('\xa0','')
#         .replace('.','')
#         .replace('fb2', '.fb2')
#          )
#     shutil.move(fn, f'{tmpfb2clean}/{nn}')
# print(f'{len(fbs)} fb2(s) sanitized')

# # In order to convert you need to get the conversion xsl from here:
# # https://github.com/mgrankin/ru_transformers/blob/master/corpus/FB2_2_txt.xsl
# # and put it in the root folder
# def convert_fb2(fn):
#     #!xsltproc FB2_2_txt.xsl {fn} > {str(fn).replace(' ', '').replace('.fb2','.txt').replace('/fb2','/txt')} 2>>/dev/null
#     !xsltproc FB2_2_txt.xsl {fn} > {tmptxt + '/' + fn.name.replace('fb2', 'txt')} 2>>/dev/null
#     return {fn}

# # Get fb2s from the clean snitized dir
# fbs = get_files(tmpfb2clean, '.fb2')

# # convert all to .txt
# thread_map(convert_fb2, fbs, max_workers=64)
# print('FB2(s) conversion done')

### Filter and concat txt files

In [ ]:
txts = get_files('./tmp/txt', '.txt')
print(f'Found {len(txts)} txt(s)')

# this will take time, bcs langdetect fails on multithreading
print('Running langdetect . . . ')
for fn in progress_bar(txts):
    with open(f'./{fn}', 'r') as f:
        lines = f.read()
        print(f)
        print(fn)
        print(len(lines))
        try:
            if len(lines) > 1e+4 and detect(lines) == 'ru':
                with open(f'{data}/{fn.name}', 'w') as c:
                    c.write(lines)
        except LangDetectException as e:
            pass

# Add space before each word. It's not really nesessary. 
# It just makes encoding a bit more meaningful to the model and the text smaller(after encoding).
print('Running text sanitization . . . ')
def process_fn(fn):
    match = re.compile(r'(?=[^ ])([\W])([\w])')
    match2 = re.compile('(.|\s)\\1\\1+')
    with open(fn, 'r') as f:
        lines = f.read()
    if lines and lines[0] != ' ': lines = ' ' + lines
    lines = match.sub(r'\g<1> \g<2>', lines)
    lines = match2.sub(r'\1'*3, lines)
    with open(fn, 'w') as c:
        c.write(lines)
        
thread_map(process_fn, txts)

In [ ]:
txts = get_files(data, '.txt'); 
print(f'Amount of txt(s): {len(txts)}')

fsorted = get_files('../data/classic', '.txt') + sorted(txts, key=lambda fn: os.path.getsize(fn))

print('Building text corpus')
sz=0
with open('./tmp/russian_corpus_for_vocab.txt', 'w') as c:
    for fn in fsorted:
        with open(fn, 'r') as f:
            sz += c.write(f.read().replace('\n', f' {NEW_LINE} ') + '\n')
            if sz > 5e+9:
                break
print('Done ./tmp/russian_corpus_for_vocab.txt')

Now a text corpus can be collected from ./tmp/russian_corpus_for_vocab.txt to use with gpt-2